In [2]:
#Imports

#Data
import pandas as pd
import numpy as np

#Date
import datetime as dt
from datetime import datetime

#Visualisation
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

#ML

#General
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix,classification_report

#knn
from sklearn.neighbors import KNeighborsClassifier





data_file='wrang_xyz_data.csv'

financial_data=['total_rev_hi_lim','tot_cur_bal','tot_coll_amt','revol_util','loan_amnt','dti','revol_bal']
loan_data=['purpose','initial_list_status','term']
emp_data=['emp_length','collections_12_mths_ex_med','acc_now_delinq','home_ownership','annual_inc','verification_status','zip_code','addr_state','delinq_2yrs','inq_last_6mths','open_acc','pub_rec','total_acc']
date_data=['earliest_cr_line']


#out=['last_pymnt_d','last_credit_pull_d','initial_list_status','recoveries','collection_recovery_fee','last_pymnt_amnt','total_pymnt','total_rec_int','int_rate','out_prncp',''total_rec_late_fee','default_ind']

features=financial_data+loan_data+emp_data+date_data
pred_data=['issue_d','grade','sub_grade']

In [3]:
def get_model_clus(data_file='wrang_xyz_data.csv',pred_value=['grade']):
 
    df = pd.read_csv('data/'+data_file,usecols=features+['issue_d']+pred_value)

    X=df.drop([pred_value[0],'issue_d'],axis=1)
    y=df[pred_value[0]]

    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

    model_clus = KNeighborsClassifier(n_neighbors=5)
    model_clus.fit(X_train,y_train)
    y_clus_pred = model_clus.predict(X_test)
    print(confusion_matrix(y_test, y_clus_pred))
    print(classification_report(y_test, y_clus_pred))

    return model_clus

model_clus=get_model_clus(pred_value=['grade'])

[[20970 12196  2722   412    44     4     0]
 [15105 29877 13859  2658   498    48     0]
 [ 6787 22436 21341  6456  1917   237    19]
 [ 2138  9616 12946  6016  2251   347    28]
 [  480  3133  6352  3984  2179   460    36]
 [   77   611  1760  1506   957   273    34]
 [    9   112   372   329   267   115    14]]
              precision    recall  f1-score   support

           0       0.46      0.58      0.51     36348
           1       0.38      0.48      0.43     62045
           2       0.36      0.36      0.36     59193
           3       0.28      0.18      0.22     33342
           4       0.27      0.13      0.18     16624
           5       0.18      0.05      0.08      5218
           6       0.11      0.01      0.02      1218

    accuracy                           0.38    213988
   macro avg       0.29      0.26      0.26    213988
weighted avg       0.36      0.38      0.36    213988



In [4]:
features=['int_rate']
def get_model_clus(data_file='wrang_xyz_data.csv',pred_value=['grade']):
 
    df = pd.read_csv('data/'+data_file,usecols=features+['issue_d']+pred_value)

    X=df.drop([pred_value[0],'issue_d'],axis=1)
    y=df[pred_value[0]]

    scaler = StandardScaler()
    scaler.fit(X)
    X = scaler.transform(X)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.75, test_size=0.25)

    model_clus = KNeighborsClassifier(n_neighbors=5)
    model_clus.fit(X_train,y_train)
    y_clus_pred = model_clus.predict(X_test)
    print(confusion_matrix(y_test, y_clus_pred))
    print(classification_report(y_test, y_clus_pred))

    return model_clus

model_clus=get_model_clus(pred_value=['sub_grade'])

[[5684   13    4 ...    0    0    0]
 [   0 5533    0 ...    0    0    0]
 [   0    0 5642 ...    0    0    0]
 ...
 [   0    0    0 ...  183    0    0]
 [   0    0    0 ...    2   41    0]
 [   0    0    0 ...    0    0  117]]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5701
           1       1.00      1.00      1.00      5533
           2       1.00      0.97      0.98      5822
           3       0.98      1.00      0.99      8498
           4       1.00      1.00      1.00     11077
           5       0.99      0.81      0.89     11093
           6       0.57      1.00      0.72     11722
           7       0.83      0.86      0.84     13788
           8       0.64      0.52      0.57     13568
           9       0.90      0.76      0.82     11897
          10       0.91      0.75      0.82     12995
          11       0.91      0.85      0.88     12740
          12       0.96      0.94      0.95     12060
          13   